In [1]:
import pandas as pd
import numpy as np

In [2]:
# Importación de datos:
dataset_path = r"~/Desktop/Mate_PI/files/"
matepi = pd.read_csv(dataset_path + "Mate_PI_2020_2025.csv", index_col=0, dtype={"Condicion":"category", "Tipo_Cursada":"category"})

# Descripción de la base de datos

Cada registro es único, corresponde a los resultados de cada alumno, identificados por el índice de fila.

- **1P1F**: nota del Primer Parcial, Primera Fecha.
- **1P2F**: nota del Primer Parcial, Segunda Fecha.
- **2P1F**: nota del Segundo Parcial, Primera Fecha.
- **2P2F**: nota del Segundo Parcial, Segunda Fecha.

Los datos de cada columna son numéricos, la celda está vacía si el alumno no se presentó a rendir. La nota mínima para aprobar cada parcial es 4, la materia se promociona con un promedio mayor o igual a 6. Contienen números tipo `float` entre 0 y 10.

- **F1**: columna numérica, nota del Flotante de Primer Parcial. Esta fecha permite recuperar el Primer Parcial si ya se logró aprobar el Segundo en las instancias anteriores. Contiene números tipo `float` entre 0 y 10.
- **F2**: columna numérica, nota del Flotante de Segundo Parcial. Esta fecha permite recuperar el Segundo Parcial si ya se logró aprobar el Primero en las instancias anteriores.  Contiene números tipo `float` entre 0 y 10.
- **Condicion**: columna categórica, indica la condición final del alumno luego de los parciales.
    -  *Libre*: no se presentó a ningún parcial.
    -  *Abandonó*: rindió algún parcial pero no agotó las instancias posibles.
    -  *Desaprobado*: no logró aprobar Primer Parcial y Segundo Parcial luego de agotar las instancias posibles, o sí lo hizo pero no logró alcanzar el promedio de Promoción.
    -  *Promocionado*: aprobó ambos parciales con un promedio mayor o igual a 6.
- **Final**: columna numérica, nota final de los alumnos promocionados, la celda está vacía en los otros casos.  Contiene números tipo `int` entre 6 y 10.
- **Año**: columna numérica, año de la cursada. Inicia en 2020.
- **Tipo_Cursada**: columna categórica, indica el período en el que se realizó la cursada.
    - *Verano*: cursada intensiva de Enero-Febrero.
    - *1er Semestre*: cursada regular de Marzo-Junio.
    - *Anticipada*: cursada regular de Agosto-Noviembre.
- **Virtual**: columna binaria, indica si la cursada fue dictada en modalidad virtual o no.
- **Oral**: columna binaria. En las cursadas virtuales, además de aprobar los parciales, se requería aprobar un examen oral. Esta columna indica los resultados como *Aprobado* o *Desaprobado*, la celda está vacía si el alumno no se presentó.
- **Grupo**: columna de texto, indica nombre de la comisión de la que se extrajeron los datos. Si la comisión es única la celda está vacía.

# Limpieza de los datos

Se realiza una revisión de cada columna, para asegurar la consistencia de los datos, corregir errores de tipeo o de tipo de dato.

In [3]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1223 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          793 non-null    float64 
 1   1P2F          493 non-null    float64 
 2   2P1F          582 non-null    float64 
 3   2P2F          310 non-null    float64 
 4   F1            80 non-null     float64 
 5   F2            77 non-null     float64 
 6   Condicion     1223 non-null   category
 7   Final         347 non-null    float64 
 8   Año           1223 non-null   int64   
 9   Tipo_Cursada  1223 non-null   category
 10  Virtual       1223 non-null   object  
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(7), int64(1), object(3)
memory usage: 117.4+ KB


## 1- Columna `Condicion`: revisión de consistencia

Debe corresponder con:

- Libre: si todas las columnas de Parciales y Flotante son nulas.
- Desaprobado: si el promedio entre 1P y 2P es menor que 6 y
    - F1 y F2 son nulas
    - F1 no nula y el promedio entre F1 y P1 es menor que 6
    - F2 no nula y el promedio entre F2 y P2 es menor que 6
- Promocionado: en otro caso 

In [4]:
# Revisión: Libre
libres = matepi.loc[matepi["Condicion"]=="Libre"]

# Cantidad de nulos en cada columna: deben ser 318 registros
libres.isna().sum()

1P1F            318
1P2F            318
2P1F            318
2P2F            318
F1              318
F2              318
Condicion         0
Final           318
Año               0
Tipo_Cursada      0
Virtual           0
Oral            318
Grupo           249
dtype: int64

Se verifica que **todas las columnas de Parciales y Flotantes son nulas para los Libres**.

----------

Pasamos a revisar los Desaprobados.

In [5]:
# Revision: Desaprobado
desaprobados = matepi.loc[ matepi["Condicion"]=="Desaprobado" ]
desaprobados

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,1.8,1.4,3.2,0.6,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1190,0.0,NaN,0.3,0.1,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1200,1.7,4.0,4.0,NaN,NaN,4.9,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1207,1.6,3.0,1.9,4.0,5.3,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN


**ERROR EN LA TOMA DE DATOS: hay filas que deberían corresponder con la categoría Libre**. Las aislamos:

In [6]:
# Error en la categoría: si todos los Parciales y Flotantes son nulos, debería ser Libre
desaprobados[desaprobados["1P1F"].isna() & desaprobados["1P2F"].isna() \
            & desaprobados["2P1F"].isna() & desaprobados["2P2F"].isna()]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
34,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
43,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN


Notamos que **se trata de los datos correspondientes al 2020: esas filas deberían corresponder con Libres**. Sim embargo, al revisar todos los registros de Desaprobados del 2020, notamos que **se trata de Desaprobados reales, el error está en la falta del registro de las notas de parciales**.

In [7]:
desaprobados[desaprobados["Año"] == 2020]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
34,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
43,NaN,NaN,NaN,NaN,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN


Disgnosticamos que **para el curso de Verano 2020 no se registraron las notas de los Desaprobados**. Sin embargo, la `Condicion` es correcta, **decido reemplazar las notas faltantes en Parciales por 0**. 

In [8]:
# Reemplazo los nulos de Parciales en Desaprobados por 0
condiciones = (matepi["Condicion"] == "Desaprobado") & (matepi["Año"] == 2020) \
                & (matepi["Tipo_Cursada"] == "Verano")

matepi.loc[condiciones,["1P1F", "1P2F", "2P1F", "2P2F"]] = matepi.loc[condiciones,["1P1F", "1P2F", "2P1F", "2P2F"]].fillna(0)

**Resuelto el problema de Desaprobados correspondientes a Verano 2020**. Podemos volver a filtrar y seguir la verificación.

In [9]:
# Vuelvo a filtrar para revision: Desaprobado
desaprobados = matepi.loc[ matepi["Condicion"]=="Desaprobado" ]
desaprobados

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
2,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
3,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
5,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
20,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,1.8,1.4,3.2,0.6,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1190,0.0,NaN,0.3,0.1,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1200,1.7,4.0,4.0,NaN,NaN,4.9,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN
1207,1.6,3.0,1.9,4.0,5.3,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN


In [10]:
# Ya que ya filtré que no hay libres, voy a reemplazar todos los nulos (ausentes) en 1P1F y 2P1F por 0:
parciales = ["1P1F", "2P1F"]
desaprobados[parciales].isna().sum()

1P1F    11
2P1F    16
dtype: int64

In [11]:
desaprobados.loc[:, parciales] = desaprobados.loc[:, parciales].fillna(0)

In [12]:
desaprobados[parciales].isna().sum()

1P1F    0
2P1F    0
dtype: int64

In [13]:
# Para las segundas fechas: si es nulo reemplazo por la nota en primera fecha
parciales_2 = ["1P2F", "2P2F"]
desaprobados[parciales_2].isna().sum()

1P2F    39
2P2F    22
dtype: int64

In [14]:
desaprobados.loc[:,"1P2F"] = desaprobados["1P2F"].fillna(desaprobados["1P1F"])
desaprobados.loc[:,"2P2F"] = desaprobados["2P2F"].fillna(desaprobados["2P1F"])

desaprobados[parciales_2].isna().sum()

1P2F    0
2P2F    0
dtype: int64

Voy a **verificar que los promedios sean menores que 6 antes del Flotante**:

In [15]:
# Agrego una columna con los promedios pre Flotante:
desaprobados.loc[:,"Promedio pre F"] = (desaprobados["1P2F"] + desaprobados["2P2F"])/2

/var/folders/rb/2q7b41vn7wv6pvjpltqddx_h0000gn/T/ipykernel_9939/2713777927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desaprobados.loc[:,"Promedio pre F"] = (desaprobados["1P2F"] + desaprobados["2P2F"])/2


In [16]:
# Reviso que los promedios sean menores que 6:
desaprobados[ desaprobados["Promedio pre F"] >= 6 ]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedio pre F
147,6.0,6.00,6.35,6.35,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,6.175
171,8.6,8.60,8.90,8.90,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,8.750
189,8.2,8.20,6.70,6.70,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.450
193,7.6,7.60,3.80,7.20,NaN,NaN,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.400
214,9.0,9.00,5.50,5.50,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,7.250
217,7.0,7.00,9.50,9.50,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,8.250
236,3.8,5.42,7.00,7.00,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,6.210
238,6.8,6.80,6.80,6.80,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,6.800
261,6.9,6.90,7.20,7.20,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,7.050
275,7.6,7.60,6.20,6.20,NaN,NaN,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,6.900


Vemos que **existen promedios de Promoción previos al Flotante, pero desaprobaron en el Oral**.

Trabajemos con los que debían rendir Flotante (promedio menor que 6)

In [17]:
desaprobados[ desaprobados["Promedio pre F"] < 6 ]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedio pre F
0,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN,0.00
2,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN,0.00
3,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN,0.00
5,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN,0.00
20,0.0,0.0,0.0,0.0,NaN,NaN,Desaprobado,NaN,2020,Verano,No,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,1.8,1.4,3.2,0.6,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,1.00
1190,0.0,0.0,0.3,0.1,NaN,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,0.05
1200,1.7,4.0,4.0,4.0,NaN,4.9,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,4.00
1207,1.6,3.0,1.9,4.0,5.3,NaN,Desaprobado,NaN,2025,1er Semestre,No,NaN,NaN,3.50


Reemplazo los valores nulos en F1 o F2 con la última nota obtenida en Primer Parcial o Segundo Parcial, para calcular el promedio final y **verificar que la nota final es menor que 6**.

In [18]:
# Reemplazo los nulos de Flotante con las últimas notas correspondientes:
desaprobados.loc[:,"F1"] = desaprobados["F1"].fillna(desaprobados["1P2F"])
desaprobados.loc[:,"F2"] = desaprobados["F2"].fillna(desaprobados["2P2F"])

# Calculo los promedios finales:
desaprobados.loc[:,"Promedio Final"] = (desaprobados["F1"] + desaprobados["F2"])/2

/var/folders/rb/2q7b41vn7wv6pvjpltqddx_h0000gn/T/ipykernel_9939/3613982398.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desaprobados.loc[:,"Promedio Final"] = (desaprobados["F1"] + desaprobados["F2"])/2


In [19]:
# Reviso que los promedios sean menores que 6
desaprobados[ desaprobados["Promedio Final"] >= 6 ]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedio pre F,Promedio Final
128,3.2,6.60,0.30,0.00,6.60,5.40,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,3.300,6.000
147,6.0,6.00,6.35,6.35,6.00,6.35,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,6.175,6.175
165,4.5,4.50,5.20,5.80,6.70,5.80,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,5.150,6.250
166,5.7,5.70,3.20,0.00,5.70,7.45,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,2.850,6.575
171,8.6,8.60,8.90,8.90,8.60,8.90,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,8.750,8.750
189,8.2,8.20,6.70,6.70,8.20,6.70,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.450,7.450
193,7.6,7.60,3.80,7.20,7.60,7.20,Desaprobado,NaN,2020,Anticipada,Si,Desaprobado,NaN,7.400,7.400
214,9.0,9.00,5.50,5.50,9.00,5.50,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,7.250,7.250
217,7.0,7.00,9.50,9.50,7.00,9.50,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,8.250,8.250
229,4.6,6.83,5.00,3.80,6.83,6.80,Desaprobado,NaN,2021,Anticipada,Sí,Desaprobado,NaN,5.315,6.815


Nuevamente, hay alumnos que **alcanzaron promedio de promoción pero desaprobaron en el Oral**

**Finalizamos la revisión de consistencia de Desaprobados.**

In [20]:
# Cantidad total de alumnos según Condicion
matepi.Condicion.value_counts()

Condicion
Abandonó        352
Promocionado    347
Libre           318
Desaprobado     206
Name: count, dtype: int64

----------

Resta **analizar que los Promocionados sean correctos**.

In [21]:
# Filtro según Condicion Promocionado:
promocionados = matepi[ matepi["Condicion"] == "Promocionado" ]
promocionados

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
1,NaN,7.0,8.2,NaN,NaN,NaN,Promocionado,8.0,2020,Verano,No,NaN,NaN
7,NaN,NaN,NaN,4.2,7.8,NaN,Promocionado,6.0,2020,Verano,No,NaN,NaN
10,8.6,NaN,7.9,NaN,NaN,NaN,Promocionado,8.0,2020,Verano,No,NaN,NaN
11,7.5,NaN,7.1,NaN,NaN,NaN,Promocionado,7.0,2020,Verano,No,NaN,NaN
12,NaN,8.3,9.5,NaN,NaN,NaN,Promocionado,9.0,2020,Verano,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,3.3,5.0,4.6,7.0,NaN,NaN,Promocionado,6.0,2025,1er Semestre,No,NaN,NaN
1212,4.0,6.3,3.2,7.9,NaN,NaN,Promocionado,7.0,2025,1er Semestre,No,NaN,NaN
1216,7.5,6.7,NaN,NaN,NaN,7.5,Promocionado,7.0,2025,1er Semestre,No,NaN,NaN
1218,4.5,NaN,3.6,7.6,NaN,NaN,Promocionado,6.0,2025,1er Semestre,No,NaN,NaN


Igual que antes, voy a **calcular los promedios antes del Flotante**:

- Reemplazo nulos por 0  en Primeras Fechas
- Reemplazo nulos de Segundas Fechas por los resultados de las Primeras
- Calculo promedios con `1P2F` y `2P2F`

In [22]:
# Completando nulos en Primeras Fechas con valor 0
promocionados.loc[:,"1P1F"] = promocionados.loc[:,"1P1F"].fillna(0)
promocionados.loc[:,"2P1F"] = promocionados.loc[:,"2P1F"].fillna(0)

# Completando nulos en Segundas Fechas con valor de la Primera
promocionados.loc[:,"1P2F"] = promocionados.loc[:,"1P2F"].fillna(promocionados["1P1F"])
promocionados.loc[:,"2P2F"] = promocionados.loc[:,"2P2F"].fillna(promocionados["2P1F"])

# Creación columna de Promedios pre Flotante
promocionados.loc[:, "Promedios pre F"] = (promocionados["1P2F"] + promocionados["2P2F"])/2

promocionados.head()

/var/folders/rb/2q7b41vn7wv6pvjpltqddx_h0000gn/T/ipykernel_9939/3255361802.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  promocionados.loc[:, "Promedios pre F"] = (promocionados["1P2F"] + promocionados["2P2F"])/2


,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedios pre F
1,0.0,7.0,8.2,8.2,NaN,NaN,Promocionado,8.0,2020,Verano,No,NaN,NaN,7.60
7,0.0,0.0,0.0,4.2,7.8,NaN,Promocionado,6.0,2020,Verano,No,NaN,NaN,2.10
10,8.6,8.6,7.9,7.9,NaN,NaN,Promocionado,8.0,2020,Verano,No,NaN,NaN,8.25
11,7.5,7.5,7.1,7.1,NaN,NaN,Promocionado,7.0,2020,Verano,No,NaN,NaN,7.30
12,0.0,8.3,9.5,9.5,NaN,NaN,Promocionado,9.0,2020,Verano,No,NaN,NaN,8.90


In [23]:
# Revisión de nulos en Parciales:
promocionados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 347 entries, 1 to 1219
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   1P1F             347 non-null    float64 
 1   1P2F             347 non-null    float64 
 2   2P1F             347 non-null    float64 
 3   2P2F             347 non-null    float64 
 4   F1               49 non-null     float64 
 5   F2               36 non-null     float64 
 6   Condicion        347 non-null    category
 7   Final            347 non-null    float64 
 8   Año              347 non-null    int64   
 9   Tipo_Cursada     347 non-null    category
 10  Virtual          347 non-null    object  
 11  Oral             71 non-null     object  
 12  Grupo            43 non-null     object  
 13  Promedios pre F  347 non-null    float64 
dtypes: category(2), float64(8), int64(1), object(3)
memory usage: 36.2+ KB


Alumnos con **nota menor a 6 deben presentarse a Flotante**. Voy a revisar esos registros.

In [24]:
# Filtro por Promedio pre Flotante. No puede haber registros con F1 y F2 vacíos:
condiciones = (promocionados["Promedios pre F"]<6) & promocionados["F1"].isna() & promocionados["F2"].isna()
promocionados.loc[ condiciones ]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedios pre F
223,4.6,6.38,5.6,5.60,NaN,NaN,Promocionado,6.0,2021,Anticipada,Sí,Aprobado,NaN,5.990
306,5.2,5.20,4.6,4.60,NaN,NaN,Promocionado,7.0,2022,Verano,No,NaN,NaN,4.900
336,5.9,5.90,5.9,5.90,NaN,NaN,Promocionado,6.0,2022,Verano,No,NaN,NaN,5.900
375,2.6,6.90,4.6,4.60,NaN,NaN,Promocionado,6.0,2022,Verano,No,NaN,NaN,5.750
381,4.7,4.70,7.1,7.10,NaN,NaN,Promocionado,6.0,2022,Verano,No,NaN,NaN,5.900
463,2.1,4.90,2.1,6.80,NaN,NaN,Promocionado,6.0,2022,1er Semestre,No,NaN,G1,5.850
502,6.9,6.90,2.6,5.05,NaN,NaN,Promocionado,6.0,2022,1er Semestre,No,NaN,G1,5.975
565,4.1,4.10,0.0,7.15,NaN,NaN,Promocionado,6.0,2022,1er Semestre,No,NaN,G7,5.625
643,5.7,5.70,1.3,5.90,NaN,NaN,Promocionado,6.0,2022,Anticipada,No,NaN,NaN,5.800
677,5.0,5.00,5.5,6.90,NaN,NaN,Promocionado,6.0,2022,Anticipada,No,NaN,NaN,5.950


**HAY UN DATO INCOSISTENTE EN LOS PROMOCIONADOS**. 
El **registro 306** aparece promocionado con nota 7 pero no rindió Flotante y su promedio es 4,9

Los demás son correctos. La nota fue redondeada a 6.

**SOLUCION: eliminar el registro 306**

In [25]:
# Elimino el registro 306 del DataFrame original matepi y de la copia promocionados
promocionados = promocionados.drop(index=306)
matepi = matepi.drop(index=306)

Finalmente, **calculo los promedios finales** considerando la nota del Flotante:

- Valores nulos en F1 son reemplazados por 1P2F
- Valores nulos en F2 son reemplazados por 2P2F

In [26]:
# Reemplazo nulos en F1 y F2
promocionados.loc[:, "F1"] = promocionados["F1"].fillna(promocionados["1P2F"])
promocionados.loc[:, "F2"] = promocionados["F2"].fillna(promocionados["2P2F"])

# Calculo promedios finales:
promocionados.loc[:, "Promedio"] = (promocionados["F1"] + promocionados["F2"])/2

In [27]:
promocionados.sort_values(by="Promedio")

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedios pre F,Promedio
565,4.1,4.1,0.0,7.15,4.1,7.15,Promocionado,6.0,2022,1er Semestre,No,NaN,G7,5.625,5.625
846,1.8,5.0,4.4,4.40,6.9,4.40,Promocionado,6.0,2023,Anticipada,No,NaN,NaN,4.700,5.650
375,2.6,6.9,4.6,4.60,6.9,4.60,Promocionado,6.0,2022,Verano,No,NaN,NaN,5.750,5.750
804,1.9,6.0,0.3,4.40,6.0,5.60,Promocionado,6.0,2023,Anticipada,No,NaN,NaN,5.200,5.800
643,5.7,5.7,1.3,5.90,5.7,5.90,Promocionado,6.0,2022,Anticipada,No,NaN,NaN,5.800,5.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,10.0,10.0,9.9,9.90,10.0,9.90,Promocionado,10.0,2024,Verano,No,NaN,NaN,9.950,9.950
472,10.0,10.0,10.0,10.00,10.0,10.00,Promocionado,10.0,2022,1er Semestre,No,NaN,G1,10.000,10.000
1136,10.0,10.0,10.0,10.00,10.0,10.00,Promocionado,10.0,2024,Anticipada,No,NaN,NaN,10.000,10.000
198,10.0,10.0,10.0,10.00,10.0,10.00,Promocionado,10.0,2020,Anticipada,Si,Aprobado,NaN,10.000,10.000


**Finalizamos la revisión de consistencia de Promocionados**

In [28]:
# Cantidad total de alumnos según Condicion
matepi.Condicion.value_counts()

Condicion
Abandonó        352
Promocionado    346
Libre           318
Desaprobado     206
Name: count, dtype: int64

## 2- Columnas de parciales y flotantes

In [29]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          806 non-null    float64 
 1   1P2F          507 non-null    float64 
 2   2P1F          595 non-null    float64 
 3   2P2F          324 non-null    float64 
 4   F1            80 non-null     float64 
 5   F2            77 non-null     float64 
 6   Condicion     1222 non-null   category
 7   Final         346 non-null    float64 
 8   Año           1222 non-null   int64   
 9   Tipo_Cursada  1222 non-null   category
 10  Virtual       1222 non-null   object  
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(7), int64(1), object(3)
memory usage: 117.3+ KB


Hay muchos valores nulos, correspondientes a alumnos que no se presentaron a rendir. Como convención:

- Para **Primer y Segundo Parcial**:
    - Ausente: asignar valor -1
    - Rindió: valores `float` mayores o iguales a 0.
- Para **Flotantes**:
    - Ausente: asignar valor -1
    - Rindió: valores `float` mayores o iguales a 0. 

In [30]:
parciales = ["1P1F", "1P2F", "2P1F", "2P2F"]
flotantes = ["F1", "F2"]

In [31]:
# Asignar valor -1 a los nulos en cada columna:
for col in parciales+flotantes:
    matepi.loc[:,col] = matepi.loc[:,col].fillna(-1)

In [32]:
print("Valores no nulos: parciales y flotantes")
for col in parciales+flotantes:
    print(col + ": ", 1222 - matepi[matepi[col] == -1].shape[0])

Valores no nulos: parciales y flotantes
1P1F:  806
1P2F:  507
2P1F:  595
2P2F:  324
F1:  80
F2:  77


Ya está **resuelto el problema de valores nulos en columnas de parciales y flotantes**.

In [33]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          1222 non-null   float64 
 1   1P2F          1222 non-null   float64 
 2   2P1F          1222 non-null   float64 
 3   2P2F          1222 non-null   float64 
 4   F1            1222 non-null   float64 
 5   F2            1222 non-null   float64 
 6   Condicion     1222 non-null   category
 7   Final         346 non-null    float64 
 8   Año           1222 non-null   int64   
 9   Tipo_Cursada  1222 non-null   category
 10  Virtual       1222 non-null   object  
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(7), int64(1), object(3)
memory usage: 117.3+ KB


## 3- Columna de nota `Final`

Alumnos no promocionados poseen registros vacíos en la columna Final. **Completo registros vacíos con el valor 0.**

In [34]:
# Asigno valor 0 a nulos de la columna Final
matepi.loc[:,"Final"] = matepi.loc[:,"Final"].fillna(0)

In [35]:
matepi[matepi["Condicion"] != "Promocionado"]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
0,0.0,0.0,0.0,0.0,-1.0,-1.0,Desaprobado,0.0,2020,Verano,No,NaN,NaN
2,0.0,0.0,0.0,0.0,-1.0,-1.0,Desaprobado,0.0,2020,Verano,No,NaN,NaN
3,0.0,0.0,0.0,0.0,-1.0,-1.0,Desaprobado,0.0,2020,Verano,No,NaN,NaN
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,Libre,0.0,2020,Verano,No,NaN,NaN
5,0.0,0.0,0.0,0.0,-1.0,-1.0,Desaprobado,0.0,2020,Verano,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,3.3,3.1,-1.0,-1.0,-1.0,-1.0,Abandonó,0.0,2025,1er Semestre,No,NaN,NaN
1217,0.0,-1.0,0.0,-1.0,-1.0,-1.0,Abandonó,0.0,2025,1er Semestre,No,NaN,NaN
1220,1.0,1.2,-1.0,-1.0,-1.0,-1.0,Abandonó,0.0,2025,1er Semestre,No,NaN,NaN
1221,1.2,0.9,0.0,-1.0,-1.0,-1.0,Abandonó,0.0,2025,1er Semestre,No,NaN,NaN


In [36]:
# Asigno tipo de dato: int
matepi["Final"] = matepi["Final"].astype("int")

**Completada la revisión de la columna Final**.

In [37]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          1222 non-null   float64 
 1   1P2F          1222 non-null   float64 
 2   2P1F          1222 non-null   float64 
 3   2P2F          1222 non-null   float64 
 4   F1            1222 non-null   float64 
 5   F2            1222 non-null   float64 
 6   Condicion     1222 non-null   category
 7   Final         1222 non-null   int64   
 8   Año           1222 non-null   int64   
 9   Tipo_Cursada  1222 non-null   category
 10  Virtual       1222 non-null   object  
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(6), int64(2), object(3)
memory usage: 117.3+ KB


## 4- Columnas de `Año` y `Tipo_Cursada`

In [38]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          1222 non-null   float64 
 1   1P2F          1222 non-null   float64 
 2   2P1F          1222 non-null   float64 
 3   2P2F          1222 non-null   float64 
 4   F1            1222 non-null   float64 
 5   F2            1222 non-null   float64 
 6   Condicion     1222 non-null   category
 7   Final         1222 non-null   int64   
 8   Año           1222 non-null   int64   
 9   Tipo_Cursada  1222 non-null   category
 10  Virtual       1222 non-null   object  
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(6), int64(2), object(3)
memory usage: 117.3+ KB


**Los registros en `Año` son correctos**, contienen números enteros iniciando en 2020 y no hay nulos.

In [39]:
matepi["Año"].unique()

array([2020, 2021, 2022, 2023, 2024, 2025])

**Los registros en `Tipo_Cursada` son correctos**, contienen las tres categorías adecuadas y no hay nulos.

In [40]:
matepi["Tipo_Cursada"].unique()

['Verano', 'Anticipada', '1er Semestre']
Categories (3, object): ['1er Semestre', 'Anticipada', 'Verano']

## 5- Columna `Virtual`

In [41]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          1222 non-null   float64 
 1   1P2F          1222 non-null   float64 
 2   2P1F          1222 non-null   float64 
 3   2P2F          1222 non-null   float64 
 4   F1            1222 non-null   float64 
 5   F2            1222 non-null   float64 
 6   Condicion     1222 non-null   category
 7   Final         1222 non-null   int64   
 8   Año           1222 non-null   int64   
 9   Tipo_Cursada  1222 non-null   category
 10  Virtual       1222 non-null   object  
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(6), int64(2), object(3)
memory usage: 117.3+ KB


No contiene valores nulos pero **presenta inconsistencias de tipeo**.

In [42]:
matepi["Virtual"].unique()

array(['No', 'Si', 'Sí'], dtype=object)

Voy a corregir el error pero además será conveniente **modificar el tipo de dato**, ya que los resultados son binarios:

- Asigno el valor 1 a los registros correspondientes a *Sí*.
- Asigno el valor 0 a los registros correspondientes a *No*.

In [43]:
# Asignación de valores numéricos:
matepi.loc[(matepi["Virtual"]=="No"), ["Virtual"]] = 0
matepi.loc[(matepi["Virtual"]=="Si"), ["Virtual"]] = 1
matepi.loc[(matepi["Virtual"]=="Sí"), ["Virtual"]] = 1

matepi["Virtual"] = matepi["Virtual"].astype("int")

Ya están **resueltas las inconsistencias y el tipo de datos en la columna `Virtual`**.

## 6- Columna `Oral`

In [44]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          1222 non-null   float64 
 1   1P2F          1222 non-null   float64 
 2   2P1F          1222 non-null   float64 
 3   2P2F          1222 non-null   float64 
 4   F1            1222 non-null   float64 
 5   F2            1222 non-null   float64 
 6   Condicion     1222 non-null   category
 7   Final         1222 non-null   int64   
 8   Año           1222 non-null   int64   
 9   Tipo_Cursada  1222 non-null   category
 10  Virtual       1222 non-null   int64   
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(6), int64(3), object(2)
memory usage: 117.3+ KB


In [45]:
matepi["Oral"].unique()

array([nan, 'Aprobado', 'Desaprobado'], dtype=object)

Los registros en esta columna debieran ser **no nulos sólo para las cursadas virtuales**. Y en ese caso corresponder a alguna de las categorías adecuadas.

In [46]:
# Cuento registros corresp. a cursadas virtuales
matepi[matepi["Virtual"]==1].count()

1P1F            187
1P2F            187
2P1F            187
2P2F            187
F1              187
F2              187
Condicion       187
Final           187
Año             187
Tipo_Cursada    187
Virtual         187
Oral             89
Grupo             0
dtype: int64

Se verifica lo anterior. Además, **sólo se debe evaluar oralmente a quienes estaban en condición de Promoción luego de los Flotantes**.

Para esa última revisión, voy a calcular los promedios finales y corroborar:

- Ausentes en Primeras Fechas: asigno el valor 0
- Ausentes en Segundas Fechas: asigno el valor de la Primera Fecha
- Ausentes en Flotantes: asigno el valor de 1P2F o 2P2F según el caso

In [47]:
# Filtro datos según columna `Virtual`
virtual = matepi.loc[matepi["Virtual"]==1]

# Asigno 0 a ausentes en Primeras Fechas;
virtual.loc[(virtual["1P1F"]==-1), "1P1F"] = 0
virtual.loc[(virtual["2P1F"]==-1), "2P1F"] = 0

# Asigno Primeras Fechas a ausentes de Segunda Fecha:
virtual.loc[(virtual["1P2F"]==-1), "1P2F"] = virtual.loc[(virtual["1P2F"]==-1), "1P1F"]
virtual.loc[(virtual["2P2F"]==-1), "2P2F"] = virtual.loc[(virtual["2P2F"]==-1), "2P1F"]

# Calculo promedios pre Flotante:
virtual.loc[:, "Promedios pre F"] = (virtual["1P2F"] + virtual["2P2F"])/2

# Asigno Segundas Fechas a ausentes de F1 o F2
virtual.loc[(virtual["F1"]==-1), "F1"] = virtual.loc[(virtual["F1"]==-1), "1P2F"]
virtual.loc[(virtual["F2"]==-1), "F2"] = virtual.loc[(virtual["F2"]==-1), "2P2F"]

# Calculo promedios fnales:
virtual.loc[:, "Promedio"] = (virtual["F1"] + virtual["F2"])/2

/var/folders/rb/2q7b41vn7wv6pvjpltqddx_h0000gn/T/ipykernel_9939/2110209534.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virtual.loc[:, "Promedios pre F"] = (virtual["1P2F"] + virtual["2P2F"])/2
/var/folders/rb/2q7b41vn7wv6pvjpltqddx_h0000gn/T/ipykernel_9939/2110209534.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virtual.loc[:, "Promedio"] = (virtual["F1"] + virtual["F2"])/2


In [48]:
# Filtro según si rindió Oral o no (valores no nulos en Oral)
virtual[virtual["Oral"].notna()].sort_values(by=["Promedio", "Año"])

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo,Promedios pre F,Promedio
223,4.6,6.38,5.60,5.60,6.38,5.60,Promocionado,6,2021,Anticipada,1,Aprobado,NaN,5.990,5.990
128,3.2,6.60,0.30,0.00,6.60,5.40,Desaprobado,0,2020,Anticipada,1,Desaprobado,NaN,3.300,6.000
237,2.9,5.23,6.20,5.50,6.50,5.50,Promocionado,6,2021,Anticipada,1,Aprobado,NaN,5.365,6.000
152,7.7,7.70,2.95,4.50,7.70,4.50,Promocionado,6,2020,Anticipada,1,Aprobado,NaN,6.100,6.100
276,7.0,7.00,5.20,5.20,7.00,5.20,Promocionado,7,2021,Anticipada,1,Aprobado,NaN,6.100,6.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,9.2,9.20,9.90,9.90,9.20,9.90,Promocionado,10,2020,Anticipada,1,Aprobado,NaN,9.550,9.550
263,9.2,9.20,10.00,10.00,9.20,10.00,Promocionado,10,2021,Anticipada,1,Aprobado,NaN,9.600,9.600
155,10.0,10.00,9.25,9.25,10.00,9.25,Promocionado,10,2020,Anticipada,1,Aprobado,NaN,9.625,9.625
240,9.5,9.50,10.00,10.00,9.50,10.00,Promocionado,10,2021,Anticipada,1,Aprobado,NaN,9.750,9.750


Se verifica que **los no nulos en `Oral` estaban en condiciones de rendir**.

A pesar de contener valores nulos, esta columna representa una situación excepcional de los años 2020 y 2021. **Decido mantener los valores de la columna como fueron registrados originalmente**.

## 7- Columna `Grupo`

In [49]:
# Revisión de categorías en la columna
matepi["Grupo"].unique()

array([nan, 'G1', 'G7'], dtype=object)

In [50]:
# Revisión de valores en otras columnas
matepi[matepi["Grupo"].notna()]

,1P1F,1P2F,2P1F,2P2F,F1,F2,Condicion,Final,Año,Tipo_Cursada,Virtual,Oral,Grupo
406,2.2,3.4,2.15,3.5,-1.0,-1.0,Desaprobado,0,2022,1er Semestre,0,NaN,G1
407,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,Libre,0,2022,1er Semestre,0,NaN,G1
408,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,Libre,0,2022,1er Semestre,0,NaN,G1
409,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,Libre,0,2022,1er Semestre,0,NaN,G1
410,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,Libre,0,2022,1er Semestre,0,NaN,G1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,Libre,0,2022,1er Semestre,0,NaN,G7
616,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,Libre,0,2022,1er Semestre,0,NaN,G7
617,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0,Libre,0,2022,1er Semestre,0,NaN,G7
618,0.0,0.0,0.00,-1.0,-1.0,-1.0,Abandonó,0,2022,1er Semestre,0,NaN,G7


La distinción por Grupo sólo se realizó durante el año 1er Semestre 2022. En ese año se registraron dos comisiones distintas, en los años restantes se registró una única comisión. **Decido mantener los valores de la columna como fueron registrados originalmente**.

# Estructura de datos luego de la limpieza

In [51]:
matepi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 0 to 1222
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   1P1F          1222 non-null   float64 
 1   1P2F          1222 non-null   float64 
 2   2P1F          1222 non-null   float64 
 3   2P2F          1222 non-null   float64 
 4   F1            1222 non-null   float64 
 5   F2            1222 non-null   float64 
 6   Condicion     1222 non-null   category
 7   Final         1222 non-null   int64   
 8   Año           1222 non-null   int64   
 9   Tipo_Cursada  1222 non-null   category
 10  Virtual       1222 non-null   int64   
 11  Oral          89 non-null     object  
 12  Grupo         214 non-null    object  
dtypes: category(2), float64(6), int64(3), object(2)
memory usage: 117.3+ KB


El dataset contiene 1222 registros.

- **1P1F**: nota del Primer Parcial, Primera Fecha. Valores `float` entre 0 y 10; si estuvo ausente se asigna -1.
- **1P2F**: nota del Primer Parcial, Segunda Fecha. Valores `float` entre 0 y 10; si estuvo ausente se asigna -1.
- **2P1F**: nota del Segundo Parcial, Primera Fecha. Valores `float` entre 0 y 10; si estuvo ausente se asigna -1.
- **2P2F**: nota del Segundo Parcial, Segunda Fecha. Valores `float` entre 0 y 10; si estuvo ausente se asigna -1.
- **F1**: nota del Flotante de Primer Parcial. Esta fecha permite recuperar el Primer Parcial si ya se logró aprobar el Segundo en las instancias anteriores. Valores `float` entre 0 y 10; si estuvo ausente se asigna -1.
- **F2**: nota del Flotante de Segundo Parcial. Esta fecha permite recuperar el Segundo Parcial si ya se logró aprobar el Primero en las instancias anteriores. Valores `float` entre 0 y 10; si estuvo ausente se asigna -1.
- **Condicion**: columna categórica, indica la condición final del alumno luego de los parciales.
    -  *Libre*: no se presentó a ningún parcial.
    -  *Abandonó*: rindió algún parcial pero no agotó las instancias posibles.
    -  *Desaprobado*: no logró aprobar Primer Parcial y Segundo Parcial luego de agotar las instancias posibles, o sí lo hizo pero no logró alcanzar el promedio de Promoción.
    -  *Promocionado*: aprobó ambos parciales con un promedio mayor o igual a 6.
- **Final**: nota final de los alumnos promocionados. Valores `int` entre 6 y 10; si no promociona se asigna 0.
- **Año**: año de la cursada. Valores `int` iniciando en 2020.
- **Tipo_Cursada**: columna categórica, indica el período en el que se realizó la cursada.
    - *Verano*: cursada intensiva de Enero-Febrero.
    - *1er Semestre*: cursada regular de Marzo-Junio.
    - *Anticipada*: cursada regular de Agosto-Noviembre.
- **Virtual**: indica si la cursada fue dictada en modalidad virtual o no. Valores `int`, asigna 1 si fue virtual y 0 si no lo fue.
- **Oral**: columna binaria. En las cursadas virtuales, además de aprobar los parciales, se requería aprobar un examen oral. Esta columna indica los resultados como *Aprobado* o *Desaprobado*, la celda está vacía si el alumno no se presentó.
- **Grupo**: columna nominal, indica nombre de la comisión de la que se extrajeron los datos. Presenta los valores *G1* y *G7* para los registros correspondientes a *1er Semestre 2022*. En los otros casos no se consigna distinción por comisiones (comisión única)

**Se exporta la base de datos corregida** a un archivo de texto, para su posterior análisis.

In [52]:
matepi.to_csv(dataset_path + "Mate_PI_full_clean.csv")